# Emerging Technologies Assessment

---
## Introduction
---

This notebook explores the difference between classical and quantum algorithms through the
Deutsch and Deutsch–Jozsa problems. These problems demonstrate how quantum computation can
provide exponential speedups over classical deterministic algorithms for certain tasks.

In particular, the task is to determine whether a Boolean function is constant or balanced.
This may require evaluating all possible inputs while quantum algorithms can solve the problem with a single oracle query.

The notebook is structured into five problems, each building on the previous to progress
from classical computation to quantum implementations using Qiskit.

---
## Problem 1: Generating Random Boolean Functions
---

The Deutsch–Jozsa algorithm operates on Boolean functions that satisfy a *promise*:
the function is either **constant**, returning the same value for all inputs, or
**balanced**, returning `True` for exactly half of all possible inputs.

For a function of four Boolean inputs, there are \(2^4 = 16\) possible input
combinations. A balanced function must therefore return `True` for exactly 8 of
these inputs and `False` for the remaining 8.

The goal of this problem is to construct a Python function that randomly selects
and returns a Boolean function satisfying this promise.

This implementation follows the promise condition described in the
Deutsch–Jozsa problem, where a Boolean function is guaranteed to be either
constant or balanced [IBM Quantum, Deutsch–Jozsa Algorithm].

In [6]:
import random

In [7]:
# Generate all possible input combinations for four Boolean variables.
# There are 2^4 = 16 total combinations.
INPUTS = []
for a in [False, True]:
    for b in [False, True]:
        for c in [False, True]:
            for d in [False, True]:
                INPUTS.append((a, b, c, d))

def constant_function(value):
    """
    Returns a constant Boolean function of four Boolean inputs.
    
     Parameters:
        value (bool): The Boolean value returned for all inputs.

    Returns:
        function: A function f(a, b, c, d) that always returns value.   
    """
    def f(a, b, c, d):
        # Ignore inputs and always return the same value
        return value
    return f

def balanced_function():
    """
    Returns a balanced Boolean function of four Boolean inputs.
    Exactly half of all possible inputs evaluate to True.
    The remaining half evaluate to False.
    """
    # Randomly select half of the input combinations to return True
    true_inputs = set(random.sample(INPUTS, k=len(INPUTS) // 2))

    def f(a, b, c, d):
        # Return True if the input tuple is in the selected set
        return (a, b, c, d) in true_inputs

    return f

### Random Selection of Constant or Balanced Functions

The `random_constant_balanced` function randomly returns either a constant or
balanced Boolean function with four inputs. The selection is performed using a
uniform random choice this ensures that both function types are equally likely.

By constructing the returned function exclusively from previously defined
constant and balanced generators. This implementation guarantees compliance
with the Deutsch–Jozsa promise condition. Metadata is attached to the returned
function to explicitly record its type, which simplifies testing and later
analysis.


In [ ]:

def random_constant_balanced():
    """
    Returns a randomly chosen Boolean function of four Boolean inputs.

    The returned function is guaranteed to be either constant or balanced this
    satisfies the Deutsch–Jozsa promise condition as defined by IBM Quantum.
    """
    # Randomly choose whether the function is constant or balanced
    if random.choice([True, False]):
        # Create a constant function returning either True or False
        value = random.choice([False, True])
        f = constant_function(value)
        f.function_type = "constant"
        return f
    else:
        # Create a balanced function
        f = balanced_function()
        f.function_type = "balanced"
        return f


### Verification of Balanced Function

To verify that the generated function is balanced, the function is evaluated on
all 2⁴ = 16 possible Boolean input combinations. The number of True and False
outputs is then counted.

A function is considered balanced if exactly half of the outputs evaluate to
True and the remaining half evaluate to False. The counts produced below
demonstrate that the balanced function generator satisfies this condition.

In [9]:
# Generate a balanced function for verification
f_bal = balanced_function()
# Evaluate the function on all possible inputs
outputs = [f_bal(*x) for x in INPUTS]
# Count True and False outputs to confirm balance
sum(outputs), len(outputs) - sum(outputs)

(8, 8)

### Testing the Random Function Generator

The function returned by `random_constant_balanced` is evaluated across all
possible Boolean input combinations. The declared function type is compared
against the observed distribution of outputs to confirm correctness.

A constant function produces either all True or all False outputs, while a
balanced function produces an equal number of True and False results. The
output counts shown below verify that the generated function behaves as
expected.


In [10]:
# Test the random_constant_balanced function

# Generate a random constant or balanced function
test_function = random_constant_balanced()
# Evaluate the function on all possible inputs
outputs = [test_function(*x) for x in INPUTS]

# Display declared type and output distribution
print("Declared type:", test_function.function_type)
print("True outputs:", sum(outputs))
print("False outputs:", len(outputs) - sum(outputs))

Declared type: constant
True outputs: 0
False outputs: 16


---
### Research Context: Constant and Balanced Boolean Functions
---

The Deutsch–Jozsa problem is defined under a *promise condition*, where the Boolean
function under investigation is guaranteed to be either constant or balanced.
For a function with four Boolean inputs, there are 2⁴ = 16 possible input
combinations.

A constant function returns the same output for all inputs, while a balanced
function returns True for exactly half of the input combinations and False for
the remaining half. This distinction is central to the Deutsch–Jozsa algorithm
as it allows the quantum solution to exploit interference effects to determine
the global property of the function.

In a classical deterministic setting, identifying whether a function is constant
or balanced requires evaluating the function on all possible inputs in the worst
case. For four inputs, this requires up to 16 function evaluations. This cost
grows exponentially with the number of inputs, scaling as O(2ⁿ).

The function generators implemented in this problem explicitly enforce the
Deutsch-Jozsa promise by construction. Balanced functions are generated by
selecting exactly half of all possible input combinations to return True, ensuring
that the promise condition is always satisfied before further classical or
quantum analysis is performed.


---
### Critical Insight: Query Complexity and Promise Enforcement
---
The Deutsch–Jozsa problem is an example of a *quantum query algorithm* where the
objective is to determine a global property of a Boolean function using the
minimum number of oracle queries. In this model, the promise that a function is
either constant or balanced is essential, as the correctness of the algorithm
depends on interference patterns that emerge only when the promise is strictly
satisfied.

By enforcing the promise condition during function generation, this
implementation ensures that any observed advantage in later quantum
experiments reflects genuine differences in query complexity rather than
artefacts of function construction or sampling error. This interpretation
follows IBM Quantum’s discussion of Deutsch–Jozsa as a canonical query problem
in the quantum query model.

**Reference:**  
IBM Quantum – *Quantum Query Algorithms*  
https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms



## Problem 2: Classical Testing for Function Type

### Problem Description

The goal of this problem is to determine whether a given Boolean function with
four Boolean inputs is constant or balanced using a classical deterministic
approach. The function is guaranteed to satisfy the Deutsch–Jozsa promise
condition this means it is either constant or balanced.

Unlike the quantum solution, a classical algorithm must rely on repeated
function evaluations to infer the global behaviour of the function.

---

### Classical Approach

A straightforward classical strategy is to evaluate the function on all
possible input combinations and record the outputs. For a function with four
Boolean inputs, there are 2⁴ = 16 possible inputs.

- If all outputs are identical, the function is constant.
- If exactly half of the outputs are True and half are False, the function is
  balanced.

This approach guarantees a correct classification, but it requires exhaustive
evaluation of the input space.

---

### Worst-Case Analysis

In the worst case, a classical deterministic algorithm must evaluate the
function on all 16 possible inputs to be certain of its classification. This is
because a function may appear constant for many evaluations before revealing a
different output.

For a Boolean function with *n* inputs, the number of possible
inputs grows exponentially as 2ⁿ. As a result, the classical deterministic
solution scales with time complexity O(2ⁿ).

---

### Research Context: Classical vs Quantum Evaluation

The exponential cost of classical evaluation highlights the motivation for
quantum query algorithms such as Deutsch–Jozsa. While a classical algorithm may
require evaluating every possible input, the Deutsch–Jozsa quantum algorithm
can determine whether a function is constant or balanced using a single oracle
query.

This contrast illustrates a fundamental difference between classical and
quantum computation: classical algorithms accumulate information through
repeated sampling but the quantum algorithms exploit superposition and
interference to extract global properties of a function efficiently.

In [ ]:
def determine_constant_balanced(f):
    """
    Determines whether a Boolean function with four Boolean inputs
    is constant or balanced using a classical deterministic approach.

    Efficiency note:
    This implementation evaluates the function on all 16 possible input
    combinations to guarantee correctness. In theory at most
    9 function calls are sufficient to be 100% certain.

    If 9 outputs are identical the function must be constant, since a
    balanced function can only produce 8 True and 8 False outputs.
    """

    # Evaluate the function on all possible inputs
    outputs = [f(*x) for x in INPUTS]

    # If all outputs are identical, the function is constant
    if all(output == outputs[0] for output in outputs):
        return "constant"

    # Otherwise, by the promise condition, the function is balanced
    return "balanced"


## Problem 3: Quantum Oracles

In [7]:
# Code cell

## Problem 4: Deutsch's Algorithm with Qiskit


In [8]:
# Code cell

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm

In [9]:
# Code cell